# VacationPy
----

#### Note
* Keep an eye on API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor usage and billing.

In [1]:
# !pip install gmaps

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from dotenv import load_dotenv

# API key setup
load_dotenv()

google_api_key = os.getenv("google_api_key")

# Import API key
# from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_df = pd.read_csv("../output_data/weather_data.csv")
del weather_df['Unnamed: 0']
weather_df

,Name,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,arraial do cabo,-22.9661,-42.0278,87.76,40.0,0.0,23.02,BR,1.630000e+09
1,jamsa,61.8642,25.1900,56.14,80.0,93.0,11.07,FI,1.630000e+09
2,busselton,-33.6500,115.3333,47.08,51.0,58.0,17.13,AU,1.630000e+09
3,abu samrah,35.3029,37.1841,87.39,27.0,0.0,8.93,SY,1.630000e+09
4,ipixuna,-1.7625,-48.8043,85.95,51.0,94.0,4.99,BR,1.630000e+09
...,...,...,...,...,...,...,...,...,...
531,grand gaube,66.7131,27.4306,46.71,73.0,79.0,2.86,FI,1.630000e+09
532,san quintin,29.0331,21.5482,89.76,23.0,0.0,9.86,LY,1.630000e+09
533,dzhusaly,43.8260,-111.7897,74.46,32.0,1.0,8.05,US,1.630000e+09
534,gorin,1.1983,-70.1733,92.01,45.0,67.0,2.77,CO,1.630000e+09


In [4]:
weather_df.max()

Name          zyryanskoye
Latitude          78.2186
Longitude         179.317
Max_Temp           105.37
Humidity              100
Cloudiness            100
Wind_Speed          27.58
Date             1.63e+09
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
locations = weather_df[['Latitude','Longitude']]
humidity = weather_df['Humidity']

figure_layout = {
    'width': '800px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# fig = gmaps.figure(layout=figure_layout)
fig = gmaps.figure(zoom_level=1.6, center=[0, 0])

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                dissipating=False, max_intensity=100,
                                point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotel_df = weather_df[(weather_df['Max_Temp']<80) &
                      (weather_df['Max_Temp']>70) &
                      (weather_df['Wind_Speed']<10) &
                      (weather_df['Cloudiness']==0)]
hotel_df['Hotel_Name'] = ""
hotel_df = hotel_df.reset_index(drop=True)
hotel_df = hotel_df.rename(columns={"Name":"City"})
hotel_df['City'] = hotel_df['City'].str.title()
hotel_df

C:\Users\srobi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
0,Atuona,23.1935,116.6142,78.69,87.0,0.0,1.14,CN,1.630000e+09,
1,Emba,-14.5428,40.6728,70.57,85.0,0.0,4.88,MZ,1.630000e+09,
2,Hithadhoo,18.2164,42.5053,73.22,68.0,0.0,4.61,SA,1.630000e+09,
3,Saryshagan,38.2061,67.1986,73.62,21.0,0.0,9.51,UZ,1.630000e+09,
4,Takoradi,20.8783,-156.6825,76.75,69.0,0.0,8.01,US,1.630000e+09,
5,Mount Isa,-17.8786,36.8883,73.35,83.0,0.0,5.75,MZ,1.630000e+09,


In [30]:
# find the first hotel result for city within 5,000 meters
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():

    # get latitude and longitude from df
    latitude = row['Latitude']
    longitude = row['Longitude']
    city_name = row['City']

    params = {
        "location": f"{latitude}, {longitude}",
        "radius": "5000",
        "type": "hotel",
        "key": google_api_key,
    }

    # assemble url and make API request
    print(f"Retrieving results for index {index}: {city_name.title()}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Hotel for city {city_name.title()} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel_Name'] = results[0]['name']
         
    except (KeyError, IndexError):
        print(f"Missing for city {city_name} result... skipping.")
        
    print("------------")

Retrieving results for index 0: Atuona.
Hotel for city Atuona is Shantou.
------------
Retrieving results for index 1: Emba.
Hotel for city Emba is Nacala.
------------
Retrieving results for index 2: Hithadhoo.
Hotel for city Hithadhoo is Abha.
------------
Retrieving results for index 3: Saryshagan.
Hotel for city Saryshagan is Baysun.
------------
Retrieving results for index 4: Takoradi.
Hotel for city Takoradi is Lahaina.
------------
Retrieving results for index 5: Mount Isa.
Hotel for city Mount Isa is Quelimane.
------------


In [8]:
hotel_df

,City,Latitude,Longitude,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
0,atuona,23.1935,116.6142,78.69,87.0,0.0,1.14,CN,1.630000e+09,Shantou
1,emba,-14.5428,40.6728,70.57,85.0,0.0,4.88,MZ,1.630000e+09,Nacala
2,hithadhoo,18.2164,42.5053,73.22,68.0,0.0,4.61,SA,1.630000e+09,Abha
3,saryshagan,38.2061,67.1986,73.62,21.0,0.0,9.51,UZ,1.630000e+09,Baysun
4,takoradi,20.8783,-156.6825,76.75,69.0,0.0,8.01,US,1.630000e+09,Lahaina
5,mount isa,-17.8786,36.8883,73.35,83.0,0.0,5.75,MZ,1.630000e+09,Quelimane


In [31]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt style="color: black; font-weight: 800; font-size: 15px">Name</dt>
<dd>{Hotel_Name}</dd>
<dt style="color: black; font-weight: 800; font-size: 15px">City</dt>
<dd>{City}</dd>
<dt style="color: black; font-weight: 800; font-size: 15px">Country</dt>
<dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Latitude", "Longitude"]]

In [32]:
locations2

,Latitude,Longitude
0,23.1935,116.6142
1,-14.5428,40.6728
2,18.2164,42.5053
3,38.2061,67.1986
4,20.8783,-156.6825
5,-17.8786,36.8883


In [33]:
# fig = gmaps.figure()

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations2,info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))